In [1]:
import pandas as pd
import numpy as np
import re
import PyPDF2 
import textwrap
import pdfplumber
from PyPDF2 import PdfReader

In [5]:
pdf_file = input("Enter file:") 



Enter file: C:\Users\SRIVAA86\OneDrive - Pfizer\Desktop\NLP\New folder\C3441021 Statistical Analysis Plan v6.3 final.pdf


In [13]:
pattern = r"^\d+\.\s([A-Z\s\s:]+)"
sections = {}
converted_sections = {}

with open(pdf_file, 'rb') as file:
    reader = PdfReader(file)
    num_pages = len(reader.pages)

    for page_number in range(num_pages):
        page = reader.pages[page_number]
        content = page.extract_text()
        matches = re.findall(pattern, content, re.MULTILINE)
       
        for match in matches:
            section_name = re.sub(r"^\d+\.\s", "", match)
            section_key = re.sub(r"\s", "", section_name.lower())  # convert to lowercase and remove spaces
            next_integer_match = re.search(r"\d+", content[content.find(match) + len(match):])
            
            if next_integer_match:
                next_integer = int(next_integer_match.group())
                sections[section_key] = next_integer
                converted_sections[section_key] = section_name.strip()

# Store the section names and integers in a dictionary
dictionary = {section: integer for section, integer in sections.items()}

# Extract start page and end page for "analysesandsummaries" section
section_start = "analysesandsummaries"
section_end = None
start_page = None
end_page = None

if section_start in sections:
    start_page = sections[section_start]
    section_index = list(sections.keys()).index(section_start)
    
    if section_index < len(sections) - 1:
        section_end = list(sections.keys())[section_index + 1]
        end_page = sections[section_end]

# Retrieve the converted section name from the converted_sections dictionary
section_end_converted = converted_sections.get(section_end)

analysis_section_start= "analysissets"
analysis_section_end = None
analysis_start_page = None
analysis_end_page = None

if analysis_section_start in sections:
    analysis_start_page = sections[analysis_section_start]
    analysis_section_index = list(sections.keys()).index(analysis_section_start)
            
    if analysis_section_index < len(sections) - 1:
        analysis_section_end = list(sections.keys())[analysis_section_index + 1]
        analysis_end_page = sections[analysis_section_end]

analysis_section_end_converted = converted_sections.get(analysis_section_end)
print(analysis_start_page)
print(analysis_end_page)

23
24


In [8]:
def extract_text_from_pdf(pdf_file, start_page, end_page):
    with pdfplumber.open(pdf_file) as pdf:
        text = ""
        for page_num in range(start_page - 1, end_page):
            page = pdf.pages[page_num]
            page = page.crop((50.0, 70.0, 612.0, 697.0))
            text += page.extract_text()
    return text

start_page = start_page
end_page = end_page

extracted_text = extract_text_from_pdf(pdf_file, start_page, end_page)
# print(extracted_text)

In [9]:
start_keyword = 'ANALYSES AND SUMMARIES'
end_keyword = section_end_converted.strip()

# Extract the section from the PDF
extracted_text = extract_text_from_pdf(pdf_file, start_page, end_page)

# print(extracted_text)

start_index = extracted_text.find('ANALYSES AND SUMMARIES')
end_index = extracted_text.find(section_end_converted.strip())
extracted_sec = extracted_text[start_index:end_index]
print(extracted_sec)

ANALYSES AND SUMMARIES
Overview of analysis presentation:
 Continuous data will be summarized using descriptive statistics (mean, median, SD,
minimum, maximum, and 2-sided 95% CI) Categorical data (including point estimates) will be summarized by frequency counts,
percentages and binomial 95% CIs using the Clopper-Pearson method.
 Kaplan-Meier estimates (time-to-event distirbutions) will be presented and displayed
graphically where appropriate, together with a summary of associated statistics
(including HRs, 2-sided 95% CIs, and 1-sided p-values).
 There will be no adjustment for multiplicity.
6.1. Part 1 Analyses
6.1.1. Primary Endpoint
Target safety events specified inTable 4will be evaluated in the Part 1 safety analysisset.
Table 4. Target Safety Events: Part 1 (Open-Label Treatment)
Hematologic Toxicity
Any of the following considered possibly or probably related to talazoparib:
 Grade 4 anemia
 Grade 4 thrombocytopenia
 Grade 4 neutropenia (ANC <500/µL or <0.5 × 109/L)
 G

In [10]:
#extracting analysis sets
analysis_start_keyword = 'ANALYSIS SETS'
analysis_end_keyword = analysis_section_end_converted.strip()

analysis_extracted_text = extract_text_from_pdf(pdf_file, analysis_start_page, analysis_end_page)
    
analysis_start_index = analysis_extracted_text.find('ANALYSIS SETS')
analysis_end_index = analysis_extracted_text.find(analysis_section_end_converted.strip())

analysis_extracted_section=analysis_extracted_text[analysis_start_index -3:analysis_end_index]
print(analysis_extracted_section)

4. ANALYSIS SETS (POPULATIONS FOR ANALYSIS)
Data from Part 1 and Part 2 will be summarized separately.
4.1. Part 1 Analysis Sets/Populations
Analysis Set Population Applicable Analysis
Safety Population All patients who receivedat least one • Safety analyses
dose of study treatment (talazoparib or • Select baseline
enzalutamide). characteristics summaries
Pharmacokinetic All patients who received at least one
Population dose of study treatment in Part 1 and • PK analyses
provided an evaluable PK sample.
CTC Evaluable All patients from the Safety Analysis
Population Set Part 1 with a baseline CTCs • CTC analyses
assessment and at least 1 postbaseline
CTCs assessment.
4.2. Part 2 Analysis Sets/Populations
All-comers Population:Analyses on the all-comers populationwill include patients
unselected for DDR status enrolled in Cohort 1.
DDR-deficientpopulation: Analysis on the DDR-deficient population will includepatients
with DDR deficiencies enrolled in Cohorts 1 and 2.
The following popula

In [12]:
#new codes to extract tables from the analysis sets section
!pip install tabula-py

In [ ]:
import tabula

In [10]:
pattern = r'^\d\.\d\.'
lines = extracted_sec.splitlines()
data = {'Key': [], 'Value': []}

for line in lines:
    match = re.findall(pattern, line)
    if match:
        data['Key'].append(match)
        data['Value'].append(line)
        
df = pd.DataFrame.from_dict(data)
df.drop('Key',axis=1,inplace=True)
print(df)

                                                Value
0                                6.1. Part 1 Analyses
1                             6.1.1. Primary Endpoint
2                          6.1.2.1. PK Concentrations
3                              6.1.2.2. PK Parameters
4                        6.1.3. Exploratory Endpoints
5                                6.2. Part 2 Analyses
6                       6.2.1. Primary Endpoint--rPFS
7                        6.2.1.1. SensitivityAnalyses
8                           6.2.2.1. Overall Survival
9          6.2.2.2. Objective Response per RECIST 1.1
10          6.2.2.3. Duration of Soft Tissue Response
11                   6.2.2.5. Time to PSA Progression
12  6.2.2.6. Time to Initiation of Cytotoxic Chemo...
13  6.2.2.7. Time to Initiation of Antineoplastic ...
14  6.2.2.8. Time to First Symptomatic Skeletal Event
15     6.2.2.9. PFS2 Based on Investigator Assessment
16  6.2.2.10. Time to Opiate use for Prostate Canc...
17                          

In [11]:
#converting df into a list
sec_head = df['Value'].tolist()
print(sec_head)

['6.1. Part 1 Analyses', '6.1.1. Primary Endpoint', '6.1.2.1. PK Concentrations', '6.1.2.2. PK Parameters', '6.1.3. Exploratory Endpoints', '6.2. Part 2 Analyses', '6.2.1. Primary Endpoint--rPFS', '6.2.1.1. SensitivityAnalyses', '6.2.2.1. Overall Survival', '6.2.2.2. Objective Response per RECIST 1.1', '6.2.2.3. Duration of Soft Tissue Response', '6.2.2.5. Time to PSA Progression', '6.2.2.6. Time to Initiation of Cytotoxic Chemotherapy', '6.2.2.7. Time to Initiation of Antineoplastic Therapy', '6.2.2.8. Time to First Symptomatic Skeletal Event', '6.2.2.9. PFS2 Based on Investigator Assessment', '6.2.2.10. Time to Opiate use for Prostate Cancer Pain', '6.2.3. PRO Endpoints', '6.2.3.1. Time to Event Endpoints', '6.2.3.2. DescriptiveSummaries and Change from Baseline', '6.2.4. PK', '6.2.5.1. CTC Conversion Rates', '6.2.5.2. Molecular Profiling', '6.2.5.3. Concordance of DDR deficiency results', '6.3. Subgroup Analyses', '6.4. Baseline and Other Summaries', '6.4.1. Baseline Summaries', '6.

In [12]:
domain_dict={'Subject-Level Analysis Dataset': 'ADSL','Baseline summaries' : 'ADSL','Physical examination':'ADPE', 'Alopecia Areata Symptom Impact Scale': 'ADAA','Subject Disposition':'ADDS','Demographics':'ADSL','AEs':'ADAE','SAEs':'ADSAE','MAEs':'ADAE', 'Bath Ankylosing Spondylitis Dis Activity': 'ADAB','Vital signs':'ADVS', "Physician's Global Assess of Dermatology": 'ADAD', 'Analysis Adverse Events': 'ADAE', 'Infusion-Rel Reactions and Immune-Rel AE': 'ADAEI', 'Ankylosing Spondylitis Work Instability': 'ADAI', 'Ankylosing Spondylitis Health Resource': 'ADAK', 'Disease Activity Markers': 'ADAM', 'Tanner Stage of Sexual Development': 'ADAN', 'Alopecia Areata Patient Priority Outcome': 'ADAP', 'Ankylosing Spondylitis Quality of Life': 'ADAQ', 'Severity of Alopecia Tool': 'ADAS', 'Akathisia Determination Scale': 'ADAW', 'BDI-II incl baseline cutoff categories': 'ADB2', 'Bone Body Composition': 'ADMKBB', 'NCI CTC Adverse Event (PRO-CTCAE)': 'ADBC', 'Baseline Disease Sites': 'ADBD', 'Bone Age X-Ray Assessment': 'ADBX', 'Functional Assessment of Cancer Therapy': 'ADCA', "Children's Dermatology Life Quality Indx": 'ADCC', 'Reactogenicity Unplanned Assessment': 'ADCECJ', 'Hemophilia Bleed Event': 'ADCEHE', 'Signs and Symptoms of Disease': 'ADSO', 'Reactogenicity Summary Diary Data': 'ADCEVD', 'Global Impression of Change': 'ADGC', 'Childhood Health Assessment': 'ADCH', 'Global Impression of Change - Modified': 'ADCI', 'Concomitant Medications Analysis Dataset': 'ADCM', 'Ulcerative Colitis History': 'ADCMUC', 'Concord Discord (RECIST) - Inv': 'ADCONC', 'Concord Discord (RECIST) - Inv Deriv': 'ADCONCD', 'Concord Discord (PCWG3) - Inv': 'ADCONCP', 'Child Health Questionnaire 50 Item': 'ADCQ', 'Columbia Sev Suicidality Rating Scale': 'ADCS', 'Cycles': 'ADCY', 'Scoring Atopic Dermatitis': 'ADDA', 'Event Date (PCWG3) - BICR': 'ADDATEPB', 'Event Date (RECIST) -Inv': 'ADDATES', 'Event Date (RECIST) - BICR': 'ADDATESB', 'Event Date (RECIST) - Inv Derived': 'ADDATESD', 'Event Date (irRECIST) - Inv': 'ADDATESI', 'Event Date (PCWG3) - Inv': 'ADDATESP', 'Disability Assessment for Dementia Scale': 'ADDB', 'Medical Device Complaints': 'ADDE', 'Dermatitis Family Impact Questionnaire': 'ADDJ', 'Derived Response - Inv': 'ADDRS', 'Disposition Analysis Dataset': 'ADDS', 'NASH Alcohol Use Disorder ID Test': 'ADDT', 'ARGUS Case Level Death': 'ADDTHC', 'ARGUS Event Level Death': 'ADDTHE', 'Protocol Deviations Analysis Dataset': 'ADDV', 'Dual X-Ray Absorptiometry': 'ADDX', 'EuroQol 5 Dim Health State Questionnaire': 'ADE5', 'Eczema Area and Severity Index': 'ADEA', 'Quantitative Ultrasound - Elastography': 'ADEB', 'SLEDAI 2K': 'ADED', 'ECG Analysis Dataset': 'ADEG','Electrocardiogram' : 'ADEG', 'Eyelash Assessment': 'ADEL', 'Entheses Assessment': 'ADEN', 'EOG Performance Status': 'ADEO', 'EU Org for Research and Treat. of Cancer': 'ADER', 'Epworth Sleepiness Scale': 'ADES', 'Exposure Analysis Dataset': 'ADEX','Treatment Exposure' : 'ADEX','Study Treatment Exposure':'ADEX', 'Eyebrow Assessment': 'ADEY', 'Reactogenicity Daily Diary Data': 'ADFACEVD', 'Fracture Assessment': 'ADFX', 'Global Impression of Severity': 'ADGE', 'Global Assess of Fn for Schizophrenia': 'ADGF', 'Patient Global Impression of Severity': 'ADGG', 'Magnetic Resonance Imaging': 'ADMR', 'Obstetrical History (GYN)': 'ADGY', 'Health Assessment Questionnaire': 'ADHA', 'Health Care Utilization': 'ADHC', 'Hauser Diary Endpoints': 'ADHD', 'Concomitant Medications':'ADCM', 'Hemophilia Joint Health Score': 'ADHJ', 'Hospital Anxiety and Depression Scale': 'ADHL', 'Hemophilia Activities List': 'ADHM', 'Hemophilia History': 'ADHP', 'Hemophilia Quality of Life': 'ADHQ', 'Alopecia Areata History': 'ADHX', 'Inflammatory Bowel Disease Questionnaire': 'ADIB', 'Incl/Excl Crit Not Met Analysis Dataset': 'ADIE', 'Brief Fatigue Inventory': 'ADIF', 'Clin. Global Impression-Alopecia Areata': 'ADIG', 'Infant Dermatitis Quality of Life Index': 'ADIL', 'Immunogenicity Sample Analysis Dataset': 'ADISDA', 'Juvenile Joint Tenderness and Swelling': 'ADJA', 'Joint and Musculoskeletal Assessment': 'ADJM', 'Joint Pain Assessment': 'ADJP', 'Ankylosing Spondylitis Swollen Joint Cnt': 'ADJS', 'Joint Tenderness and Assessment': 'ADJT', 'Kansas City Cardiomyopathy Questionnaire': 'ADKC', 'Kellgren Lawrence Grading System': 'ADKL', 'Laboratory Analysis Dataset': 'ADLB', 'Leeds Enthesitis Index': 'ADLD', 'Lower Extremity Activity Scale': 'ADLE', 'Dermatology Life Quality Index': 'ADLI', 'Low Back Pain Stratification': 'ADLP', 'MD Anderson': 'ADMA', 'Brief Pain Inventory': 'ADMB', "Unified Parkinson's Disease Rating Scale": 'ADMD', 'Maastricht AS Entheses': 'ADME', 'Medical History Analysis Dataset': 'ADMH', 'Mobility Measurement': 'ADMM', 'Musculoskeletal History': 'ADMS', 'Muscle Strength Assessments': 'ADMU', 'North Star Ambulatory Assessment': 'ADNA', 'Neurological Examination': 'ADNE', 'Numeric Pain Scale Rating': 'ADNR', 'Neuropathy Impairment Score': 'ADNS', 'NASH Symptom Diary': 'ADNU', 'NASH Check': 'ADNZ', 'Medical Outcomes Study': 'ADOM', 'Obstetric Examination': 'ADON', 'AE': 'ADAE','SAE':'ADSAE','WOMAC': 'ADOO', 'Opioid-Related Symptom Distress Scale': 'ADOR', '8-Point Ordinal Scale': 'ADP8', 'Pediatric ACR Response and Disease Flare': 'ADPA', 'PK Concentration': 'ADPC', 'PK Concentration (Second level)': 'ADPC2', 'Pharmacodynamics Analysis Dataset': 'ADPD', 'Physical Examination Analysis Dataset': 'ADPE', "Parkinson's Disease Fatigue Scale": 'ADPF', 'Patient Health Questionnaire (PHQ)': 'ADPH', 'Patient Global Impression of Change': 'ADPI', 'Patient-Oriented Eczema Measure (POEM)': 'ADPM', 'PainDETECT Questionnaire (PDEQ)': 'ADPN', 'PK Parameters Analysis Dataset': 'ADPP', 'Secondary PK Parameters': 'ADPP2', "Parkinson's Disease Questionnaire": 'ADPQ', 'Procedures Analysis Dataset': 'ADPR', 'Psoriasis Area and Severity Index (PASI)': 'ADPS', 'Prostate Specific Antigen': 'ADPSA', 'Parental Assessment of Disease (PDIS)': 'ADPT', 'Pruritus and Symptom Assessment for AD': 'ADPU', '5-D Pruritus Scale (P5D)': 'ADPV', "Physician's Withdrawal Checklist (PWC)": 'ADPW', 'Pregnancy Outcome Questionnaire': 'ADPY', 'Quebec Task Force Classification': 'ADQF', "IC Disorder in Parkinson's Disease Scale": 'ADQI', 'Short-Form MCGILL Pain Questionnaire-2': 'ADQSFM', 'Fatigue Severity Scale': 'ADQSFS', 'Roland-Morris Disability Questionnaire': 'ADRD', 'Modified Schobers Test': 'ADRF', 'Range of Motion Assessment': 'ADRM', 'Routine Assessment of Patient Data': 'ADRO', 'Reprod System Findings Analysis Dataset': 'ADRP', 'Response (RECIST) - Inv Derived': 'ADRSD', 'Response (RECIST) - BICR': 'ADRSB', 'Response (irRECIST) - Inv': 'ADRSI', 'Response (PCWG3) - Inv': 'ADRSP', 'Response (PCWG3) - BICR': 'ADRSPB', 'PT-Reported Trt Impact Assess-Modified': 'ADRT', 'Self-Administered PT Satisfaction Scale': 'ADSA', 'Serious Adverse Events Analysis Dataset': 'ADSAE', 'ARGUS Case Level SAE': 'ADSAEC', 'ARGUS Event Level SAE': 'ADSAEE', 'Study Body Site': 'ADSB', 'Sexual Function Questionnaire (SF-28)': 'ADSC', 'Body Surface Area for Dermatitis': 'ADSD', 'PT Global Assessment of Disease Activity': 'ADSG', 'Skeletal Events': 'ADSKL', 'Shoulder Pain and Disability Index': 'ADSP', 'Short-Form 36 Health Survey Quest': 'ADSQ', 'Follow-up Contact': 'ADSSFC', 'Survey of Autonomic Symptoms': 'ADST', 'Substance Use Analysis Dataset': 'ADSU', 'Simpson Angus Rating Scale (version 2)': 'ADSW', 'Transfusion (Blood Fluids) Details': 'ADTB', 'Tuberculosis Challenge and Reaction': 'ADTC', 'Tumor Measurement (RECIST) - Inv': 'ADTR', 'Tumor Measurement (RECIST) - BICR': 'ADTRB', 'Tumor Measurement (PCWG3) - Inv': 'ADTRP', 'Tumor Measurement (PCWG3) - BICR': 'ADTRPB', 'Trt Satisfaction Quest Medicine v.II': 'ADTS', 'Taste Questionnaire (TQ)': 'ADTT', 'Time-to-Event (RECIST) - Inv': 'ADTTE', 'Time-to-Event (RECIST) - BICR': 'ADTTEB', 'Time-to-Event (RECIST) - Inv Derived': 'ADTTED', 'Time-to-Event (irRECIST) - Inv': 'ADTTEI', 'Time-to-Event (Other Oncology Endpoints)': 'ADTTEOP', 'Time-to-Event (PCWG3) - Inv': 'ADTTEP', 'Time-to-Event (PCWG3) - BICR': 'ADTTEPB', 'Time-to-Event (PSA)': 'ADTTESP', 'Mini Mental Status Examination': 'ADU2', 'UC Endoscopic Index of Severity': 'ADUE', 'Pedia Ulcerative Colitis Activity Index': 'ADUI', 'Alopecia Areata Resource Utilization': 'ADUR', 'Immunogenicity Analysis Dataset': 'ADVA', 'NASH Promis - Fatigue': 'ADVF', 'Visual Analogue Scale': 'ADVL', 'Vital Signs Analysis Dataset': 'ADVS', 'Vaccine Administration': 'ADVX', 'Walking Distance Assessment': 'ADWK', 'WOMAC Phys Funct and Stiffness Subscale': 'ADWN', 'Work Prod and Activity Impairment Quest': 'ADWP'}

In [65]:
#function to remove the numbers and special characters from the headers
# def remove_dig_char(strings):
#     pattern = r'[^a-zA-Z\s]'
#     header = []
#     for string in strings:
#         new_string = re.sub(pattern, '', string)
#         header.append(new_string)
#     return header

# header = remove_dig_char(sec_head)

In [13]:
#creating a dictionary with mapped headers and domains
from fuzzywuzzy import fuzz
unique=[]
created_dict={}
for key in sec_head:
    y=[]
    for i in domain_dict:
        
        if fuzz.token_set_ratio(i.lower(),key.lower()) > 70:
#             print(key,domain_dict[i])
            if key not in unique:
                y=[domain_dict[i]]
                unique.append(key)
            else:
                y.append(domain_dict[i])
              
        if y!=[]:        
            created_dict[key]=y  
            

C:\Anaconda3\lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [14]:
#function to retain only the unique domain types
def extract_unique_values(dictionary):
    unique_values = {}
    for key, values in dictionary.items():
        unique_values[key] = list(set(values))
    return unique_values

unique_values = extract_unique_values(created_dict)

In [15]:
domain_type = {}
for key, values in unique_values.items():
    domain_type[key] = values

In [16]:
domain_type #final output

{'6.1.2.1. PK Concentrations': ['ADPC'],
 '6.1.2.2. PK Parameters': ['ADPP2', 'ADPP'],
 '6.2.2.2. Objective Response per RECIST 1.1': ['ADRSB', 'ADRSD'],
 '6.2.2.5. Time to PSA Progression': ['ADTTESP'],
 '6.2.2.8. Time to First Symptomatic Skeletal Event': ['ADTTESP', 'ADTTEP'],
 '6.2.2.9. PFS2 Based on Investigator Assessment': ['ADEY', 'ADEL'],
 '6.2.3.1. Time to Event Endpoints': ['ADTTEOP', 'ADTTESP', 'ADTTEP'],
 '6.4. Baseline and Other Summaries': ['ADSL'],
 '6.4.1. Baseline Summaries': ['ADSL'],
 '6.4.2. Medical History': ['ADMH'],
 '6.4.4. Study Conduct and Patient Disposition': ['ADDS'],
 '6.4.5. Protocol Deviations': ['ADDV'],
 '6.4.7. Concomitant Medications': ['ADCM'],
 '6.5.1. Adverse Events': ['ADAE', 'ADSAE'],
 '6.5.2. Adverse Events of Special Interest': ['ADAE'],
 '6.5.5. Vital Signs': ['ADVS']}

In [17]:
a_df = pd.DataFrame(columns=['Key', 'Value'], dtype=object)
analysis_pattern = r'^\d\.\d\.'
analysis_lines = analysis_extracted_section.splitlines()

for aline in analysis_lines:
    analysis_matches = re.findall(analysis_pattern, aline)
    if analysis_matches != []:
        a_match = re.findall(r'Population', aline)
        if a_match != []:
            a_data = {'Key': analysis_matches, 'Value': aline}
            a_df = a_df.append(a_data, ignore_index=True)
        as_match = re.findall(r'Analysis Set', aline)
        if as_match != []:
            a_data = {'Key': analysis_matches, 'Value': aline}
            a_df = a_df.append(a_data, ignore_index=True)

a_df.drop('Key', axis=1, inplace=True)
a_df['Value'] = a_df['Value'].str.replace(r'\d\.\d\.\d\. ', '')

# Convert the 'Value' column to a list
population_list = a_df['Value'].tolist()
unique_populations = list(set(population_list))

# print(unique_populations)

C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:18: FutureWarning: The default value of regex will change from True to False in a future version.


In [18]:
def extract_tables_from_pdf(pdf_file, keyword):
    tables = []
    with pdfplumber.open(pdf_file) as pdf:
        for i, page in enumerate(pdf.pages):
            text = page.extract_text()
            if keyword in text:
                for table in page.extract_tables():
                    table_df = pd.DataFrame(table[1:], columns=table[0],dtype=object)
                    table_df.columns = table_df.columns.str.lower()
                    if keyword.lower() in table_df.columns:
                            # Check if the header of the table matches the keyword
                        if len(tables) > 0 and tables[-1].columns.equals(table_df.columns):
                                # Check if the table from the next page has the same number of columns
                            next_page = pdf.pages[i+1].extract_tables()
                            if len(next_page) > 0 and len(next_page[0][0]) == len(table_df.columns):
                                next_table_df = pd.DataFrame(next_page[0][1:], columns=next_page[0][0])
                                next_table_df = next_table_df.rename(columns=dict(zip(next_table_df.columns, tables[-1].columns)))
                                tables[-1] = pd.concat([tables[-1], next_table_df], ignore_index=True)
                                    # st.write(tables)
                                    # st.write(next_table_df)
                            else:
                                tables.append(table_df)
                        else:
                            tables.append(table_df)
                            # st.write(tables)
    return tables


In [19]:
keyword = "population"
table_dfs = extract_tables_from_pdf(pdf_file, keyword)    
if table_dfs !=[]:
    df_big = pd.concat(table_dfs, ignore_index=True)
else:
    df_big = None
        
keyword = "analysis set"
a_table_dfs = extract_tables_from_pdf(pdf_file, keyword)
if a_table_dfs !=[]:
    a_df_big = pd.concat(a_table_dfs, ignore_index=True)
else:
    a_df_big = None

keyword = "participant analysis set"
a1_table_dfs = extract_tables_from_pdf(pdf_file, keyword)

if a1_table_dfs !=[]:
    a1_df_big = pd.concat(a1_table_dfs, ignore_index=True)
else:
    a1_df_big = None
   

In [20]:
if df_big is not None or a_df_big is not None:
    final_df_big=pd.concat([df_big,a_df_big],ignore_index=True)
    word_list = []
    final_df_big.columns = final_df_big.columns.str.lower()
    if 'population' in final_df_big.columns:
        for index, row in final_df_big.iterrows():
            word_list = final_df_big['population'].str.lower().tolist()
    if 'analysis set' in final_df_big.columns:
        for index, row in final_df_big.iterrows():
            word_list = final_df_big['analysis set'].str.lower().tolist()
    if 'participant analysis set' in final_df_big.columns:
        for index, row in final_df_big.iterrows():
            word_list = final_df_big['participant analysis set'].str.lower().tolist()

    clean_word_list = [string.replace('\uf02d', '-') for string in word_list]
    table_unique_populations = [string.replace('\n', ' ') for string in clean_word_list]
                
#     print(table_unique_populations)

In [21]:
population_final=unique_populations + table_unique_populations
unique_population_final=list(set(population_final))
print(unique_population_final)

['4.1. Part 1 Analysis Sets/Populations', 'patient-reported outcome population', 'safety population', 'ctc evaluable population', '4.2. Part 2 Analysis Sets/Populations', 'pharmacokinetic population']


In [22]:
def print_matching_word(text_to_test, clean_word_list1):

    for word in clean_word_list1:

        match = re.search(r'\b{}\b'.format(word.lower()), text_to_test.lower())

        if match:

            match_out=match.group()

            print("Match found:",match_out)

            new_text =text_to_test.lower()

            start_index = new_text.index(match_out)
            end_index = start_index + len(match_out) - 1

            print("Start index:", start_index)
            print("End index:", end_index)
            


In [23]:
print_matching_word(analysis_extracted_section, unique_population_final)

Match found: 4.1. part 1 analysis sets/populations
Start index: 103
End index: 139
Match found: safety population
Start index: 185
End index: 201
Match found: 4.2. part 2 analysis sets/populations
Start index: 675
End index: 711


In [24]:
#creating dataframe for domain tagging
pattern = r'^\d+\.'

lines = extracted_sec.splitlines()
df = pd.DataFrame(columns=['Key','Value','Paragraph'])
paragraph=""

for i, line in enumerate(lines):
    matches = re.findall(pattern, line)
    if matches:
        if paragraph:
            next_paragraph = ""
            for j in range(i+1, len(lines)):
                if lines[j].strip():
                    next_paragraph = lines[j].strip()
                    break
            wrapped_lines = textwrap.wrap(paragraph.rstrip('\r\n'), width=70)
            if len(wrapped_lines) > 1:
                wrapped_lines = wrapped_lines[:-1]
            paragraph = "\n".join(wrapped_lines)
            data = {'Key': matches, 'Value': line, 'Paragraph': paragraph + " " + next_paragraph}
            df = df.append(data, ignore_index=True)
            paragraph = ""
        else:
            data = {'Key': matches, 'Value': line, 'Paragraph': " "}
            df = df.append(data, ignore_index=True)
    else:
        paragraph += " " + line.strip()
        pattern = r'^\d\.\d\.'
if paragraph:
    wrapped_lines = textwrap.wrap(paragraph.rstrip('\r\n'), width=70)
    if len(wrapped_lines) > 1:
        wrapped_lines = wrapped_lines[:-1]
    paragraph = "\n".join(wrapped_lines)
    data = {'Paragraph': paragraph}
    df = df.append(data, ignore_index=True)

df.drop('Key', axis=1,inplace=True)
df['Paragraph'] = df['Paragraph'].shift(-1)
df = df[:-1]

df

,Value,Paragraph
0,6.1. Part 1 Analyses,
1,6.1.1. Primary Endpoint,Target safety events specified inTable 4will ...
2,6.1.2.1. PK Concentrations,"In Part 1, analyses of PK concentrations will..."
3,6.1.2.2. PK Parameters,"In estimating the PK parameters, BLQ values a..."
4,6.1.3. Exploratory Endpoints,"For CTC counts, a summary of the mean (standa..."
5,6.2. Part 2 Analyses,
6,6.2.1. Primary Endpoint--rPFS,The primary efficacy analysis will compare rP...
7,6.2.1.1. SensitivityAnalyses,The following sensitivity analyses will be pe...
8,6.2.2.1. Overall Survival,OS is defined as the time from randomization ...
9,6.2.2.2. Objective Response per RECIST 1.1,ORR is defined as the proportion of patients ...


In [25]:
extracted_sec1 = ""

# Iterate over rows and append the values and corresponding paragraphs to the variable
for index, row in df.iterrows():
    value = row['Value']
    paragraph = row['Paragraph']

    # Append value and paragraph to the variable
    extracted_sec1 += f"{value}\n {paragraph}\n\n"

# Print the variable (optional)
print(extracted_sec1)

6.1. Part 1 Analyses
  

6.1.1. Primary Endpoint
  Target safety events specified inTable 4will be evaluated in the Part
1 safety analysisset. Table 4. Target Safety Events: Part 1 (Open-
Label Treatment) Hematologic Toxicity Any of the following considered
possibly or probably related to talazoparib:  Grade 4 anemia  Grade
4 thrombocytopenia  Grade 4 neutropenia (ANC <500/µL or <0.5 × 109/L)
 Grade 3 thrombocytopenia associated with clinically significant
bleeding  Grade 3 neutropenia or Grade 3 thrombocytopenia if daily
dosing is interrupted for 7days  Febrile neutropenia, defined as ANC
<1000/µLwith a single temperature of >38.3C (>101F) or a temperature
of >38C (100.4F) sustained over a 1-hour period  Neutropenic
infection (ANC <1000/µL or <1.0 × 109/L) Nonhematologic Toxicity Any
Grade 3 adverse eventconsidered possibly or probably related to
talazoparib excluding the following:  Grade 3 laboratory
abnormalities not considered clinically significant  Grade 3 adver

In [33]:
def tag_text_with_values(dictionary, text):
    tagged_text = text

    for key, values in dictionary.items():
        for value in values:
            tagged_text = tagged_text.replace(value, f"**{key}**")

    return tagged_text

In [34]:
tagged_text = tag_text_with_values(domain_type, extracted_sec1)

# Print the tagged text
print(tagged_text)

6.1. Part 1 Analyses
  

6.1.1. Primary Endpoint
  Target safety events specified inTable 4will be evaluated in the Part
1 safety analysisset. Table 4. Target Safety Events: Part 1 (Open-
Label Treatment) Hematologic Toxicity Any of the following considered
possibly or probably related to talazoparib:  Grade 4 anemia  Grade
4 thrombocytopenia  Grade 4 neutropenia (ANC <500/µL or <0.5 × 109/L)
 Grade 3 thrombocytopenia associated with clinically significant
bleeding  Grade 3 neutropenia or Grade 3 thrombocytopenia if daily
dosing is interrupted for 7days  Febrile neutropenia, defined as ANC
<1000/µLwith a single temperature of >38.3C (>101F) or a temperature
of >38C (100.4F) sustained over a 1-hour period  Neutropenic
infection (ANC <1000/µL or <1.0 × 109/L) Nonhematologic Toxicity Any
Grade 3 adverse eventconsidered possibly or probably related to
talazoparib excluding the following:  Grade 3 laboratory
abnormalities not considered clinically significant  Grade 3 adver

In [29]:
import pickle
import spacy

In [26]:
nlp = pickle.load(open('C:/Users/SRIVAA86/OneDrive - Pfizer/Desktop/NLP/ner_nlp_70_30_10_New_oversample.pkl','rb'))

In [27]:
from nltk.tokenize import sent_tokenize
import nltk
from nltk.stem import WordNetLemmatizer

lemmatizer = WordNetLemmatizer()

In [30]:
# text = ""
# sent_tokenize
sentence = sent_tokenize(extracted_sec1)
for sen in sentence:
    word_list = nltk.word_tokenize(sen)
    lem_line= ' '.join([lemmatizer.lemmatize(w.lower()) for w in word_list])
    doc=nlp(lem_line)
    color={'R_T':'pink', 'R_L':'cyan', 'R_F':'violet'}
    options={'ents':['R_T','R_L','R_F'],'colors':color}
    spacy.displacy.render(doc,style='ent',jupyter=True,options=options)

C:\Anaconda3\lib\site-packages\spacy\displacy\__init__.py:215: UserWarning: [W006] No entities to visualize found in Doc object. If this is surprising to you, make sure the Doc was processed using a model that supports named entity recognition, and check the `doc.ents` property manually if necessary.
  warnings.warn(Warnings.W006)
